# 1.Import the MountainCar environment

In [1]:
import gym

In [2]:
env = gym.make('MountainCar-v0')

print("Shape of state = ", env.observation_space.shape[0])
print("No of actions =", env.action_space.n)


Shape of state =  2
No of actions = 3


# 2.  Try some random moves to provide a baseline

In [3]:
episodes = 5
cumulative_reward = 0

for e in range(1,episodes+1):

    state = env.reset()
    done = False

    total_reward = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        print(">",action, end='')
        new_state, reward, done, info = env.step(action)
        total_reward += reward
    
    print(f'\n Episode {e} Total Reward = {total_reward}')
    cumulative_reward += total_reward
    
average_reward = cumulative_reward/episodes 
print('Average Reward =',average_reward )

> 2> 0> 1> 2> 1> 1> 1> 1> 2> 1> 2> 0> 0> 0> 0> 2> 2> 1> 1> 2> 1> 2> 1> 0> 0> 1> 2> 2> 1> 0> 1> 1> 2> 1> 1> 1> 2> 0> 0> 1> 2> 1> 0> 1> 0> 0> 1> 1> 0> 0> 1> 2> 1> 2> 1> 2> 1> 1> 0> 2> 1> 0> 2> 0> 1> 0> 0> 0> 0> 0> 1> 2> 2> 1> 1> 0> 0> 0> 1> 1> 0> 1> 2> 0> 2> 0> 2> 0> 0> 0> 2> 0> 2> 1> 1> 0> 0> 1> 2> 2> 0> 0> 2> 0> 0> 2> 0> 1> 2> 2> 1> 0> 2> 0> 1> 0> 2> 1> 1> 2> 0> 1> 0> 1> 1> 2> 0> 1> 0> 0> 2> 1> 0> 1> 2> 0> 2> 1> 0> 2> 0> 1> 1> 2> 0> 1> 2> 2> 1> 1> 1> 1> 1> 2> 1> 0> 2> 1> 2> 2> 2> 0> 0> 1> 2> 0> 0> 2> 2> 1> 2> 1> 1> 0> 2> 0> 0> 1> 1> 0> 0> 1> 0> 2> 1> 2> 0> 2> 0> 0> 2> 1> 1> 0> 1> 0> 0> 2> 2> 0
 Episode 1 Total Reward = -200.0
> 2> 1> 0> 1> 1> 2> 2> 2> 1> 0> 1> 0> 0> 0> 2> 0> 1> 0> 1> 2> 0> 0> 1> 0> 2> 2> 1> 2> 2> 0> 2> 2> 1> 2> 1> 2> 0> 2> 0> 2> 2> 2> 1> 0> 2> 1> 2> 1> 0> 1> 2> 2> 1> 0> 0> 1> 0> 0> 0> 2> 1> 2> 0> 1> 1> 2> 1> 2> 2> 2> 2> 1> 1> 2> 1> 2> 2> 2> 0> 0> 0> 0> 0> 2> 0> 0> 2> 0> 0> 1> 2> 0> 1> 2> 1> 1> 2> 0> 0> 2> 0> 2> 1> 1> 2> 1> 0> 0> 1> 0> 2> 1> 0> 0> 1> 1> 2> 0> 0> 0> 1> 1

In [4]:
# if env.viewer: env.viewer.close() # close the visulisation

# 3. Use the DQN model for the CartPole example

In [5]:
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop

from rl.agents.dqn import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import EpsGreedyQPolicy,LinearAnnealedPolicy

In [6]:

def create_agent(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape = (1, states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model
  
model = create_agent(env.observation_space.shape[0], env.action_space.n)

In [7]:
env.reset()

array([-0.4463994,  0.       ], dtype=float32)

In [8]:
memory = SequentialMemory(limit=100000, window_length=1)

#policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', 
                              #value_max=1., value_min=.1, value_test=.05, nb_steps=10000)

policy = EpsGreedyQPolicy()

dqn = DQNAgent(model=model, nb_actions=env.action_space.n, policy=policy,
               memory=memory, nb_steps_warmup=100, gamma=0.99, 
               enable_double_dqn= True,target_model_update= 1e-3 )

dqn.compile(RMSprop(lr=1e-3), metrics=['mae'])

C:\Users\zuhay\anaconda3\lib\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:

dqn.fit(env, nb_steps=100000, visualize=True, verbose=1)

Training for 100000 steps ...
Interval 1 (0 steps performed)
    3/10000 [..............................] - ETA: 5:30 - reward: -1.0000 

C:\Users\zuhay\anaconda3\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  419/10000 [>.............................] - ETA: 5:41 - reward: -1.0000

In [ ]:
env.reset()
res = dqn.test(env, nb_episodes=5, visualize=True)

In [ ]:
res.history

In [ ]:
import numpy as np
print(np.average(res.history['episode_reward']))